In [1]:
import numpy as np
import pandas as pd
import wrangle_products as w
import re
import nltk

In [2]:
products = w.wrangle_products()

Opening local amazon_cycling.csv file


In [3]:
products.head(2)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
0,QUXIS Portable High Pressure Foot Air Pump Hea...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/QUXIS-Portable-Activated...,4.0,7635,5.99,15.59,quxis portable high pressure foot air pump hea...,9.6,30540.0
1,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.0,33013.2


In [4]:
sub_cat_list = ['Tires and Tubes', 'Gloves', 'Helmets', 'Bike Stands',
       'Mountain Bikes', 'Hydration Packs', 'Jerseys', 'Fenders',
       'Cleaners', 'Socks', 'Caps', 'Touring Bikes', 'Bottles and Cages',
       'Vests', 'Road Bikes', 'Bike Racks', 'Shorts']

In [32]:
cat_list = w.get_cat_list(sub_cat_list, extra_words=['bike', 'bikes'])

### To find the product categories: we will loop through the product list, then on each product, we can loop through the cat_list, and check for matches using regex. If we find a match, we will return the product row index number, and category number based on position in the cat_list. We should also account for products having no matches or many category matches

#### lets start by figuring out the regex match formula for one cat and one row

#### regexp = r'.*{cat}.*'
regex doesn't seem to be able to handle dynamic f string style commands so lets build a table with the original category name, the lemmatized version. along with the regexp to find the lemmatized cat name

In [33]:
cat_df = pd.concat([
    pd.Series(sub_cat_list), 
    pd.Series(cat_list)
    ], axis=1).\
    rename(columns={0:'shop_cat', 1:'lemmed_cat'})

In [34]:
cat_df

,shop_cat,lemmed_cat
0,Tires and Tubes,tire tube
1,Gloves,glove
2,Helmets,helmet
3,Bike Stands,stand
4,Mountain Bikes,mountain
5,Hydration Packs,hydration pack
6,Jerseys,jersey
7,Fenders,fender
8,Cleaners,cleaner
9,Socks,sock


In [27]:
regexp = r'.*tire.*|.*tube.*'

In [30]:
regexp = r'.*horn.*|.*portable.*'

In [31]:
re.search(regexp, products.name_preped[0])

<re.Match object; span=(0, 108), match='quxis portable high pressure foot air pump heavy >

In [ ]:
expressions = []
for cat in cat_df.lemmed_cat:
    if char(' ').count() > 0:
        expression = [f"r'.*{cat}.*'"]

In [ ]:
# for product in products:
    
# cycle through cat_list
# for cat in cat_list